# Chapter 5: Transportation, Transshipment, and Assignment Problems


Characteristics of the Transportation Model: 
* A product is transported from several sources to a number of destinations at the **minimum** possible cost.
* Each source can **supply** a fixed number of units of the product, and each destination has a fixed **demand** for the product.
* The linear programming model has constraints for supply at each source and demand at each destination.
* All constraints are equalities in a balanced transportation model where **supply equals demand**.
* Constraints contain inequalities in **unbalanced** models where **supply does not equal demand**.


## Transportation Model Example
How many tons of wheat to transport from each factory to each city on a monthly basis in order to **minimize** the total cost of transportation?

Factory | Supply | City  | Demand
-|-|-|-
1. Kansas City | 150 | A. Chicago | 220
2. Omaha | 175 | B. St. Louis | 100
3. Des Moines | 275 | C. Cincinnati | 300
 Total | 600 | Total |600

Transport Cost from factory to city ($/ton):

Factory | A. Chicago | B. St. Louis  | C. Cincinnati
-|-|-|-
1. Kansas City | 6 | 8 | 10
2. Omaha | 7 | 11 | 11
3. Des Moines | 4 | 5 | 12

The LP transportation model is the following:
\begin{align}
&\text{min}\\
&\qquad z=6x_{1A}+8x_{1B}+10x_{1C}
+7x_{2A}+11x_{2B}+11x_{2C}
+4x_{3A}+5x_{3B}+12x_{3C}\\
&\text{s.t.}\\
&\qquad x_{1A}+x_{1B}+x_{1C} = 150\\
&\qquad x_{2A}+x_{2B}+x_{2C} = 175\\
&\qquad x_{3A}+x_{3B}+x_{3C} = 275\\
&\qquad x_{1A}+x_{2A}+x_{3A} = 200\\
&\qquad x_{1B}+x_{2B}+x_{3B} = 100\\
&\qquad x_{1C}+x_{2C}+x_{3C} = 300\\
&\qquad x_{ij} \ge 0~ \text{and integer}\\
\end{align}  



In [7]:
!pip install gurobipy

# Import gurobi library
from gurobipy import * # This command imports the Gurobi functions and classes.

import numpy as np

# Transportation Model Example
c = [6,8,10,7,11,11,4,5,12]    
A = [[1,1,1,0,0,0,0,0,0],
     [0,0,0,1,1,1,0,0,0],
     [0,0,0,0,0,0,1,1,1],
     [1,0,0,1,0,0,1,0,0],
     [0,1,0,0,1,0,0,1,0],
     [0,0,1,0,0,1,0,0,1]]
b =  [150,175,275,200,100,300]
print(np.array(c).shape,np.array(A).shape,np.array(b).shape)
decision_variables = range(len(c))     
constraints = range(np.array(A).shape[0])

m = Model("example")

x = []
for i in decision_variables:
    x.append(m.addVar(lb = 0, vtype = GRB.INTEGER, name = 'x' + str(i)))

m.setObjective(quicksum(c[i] * x[i] for i in decision_variables) , GRB.MINIMIZE) 
m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           == b[j] for j in constraints), "constraints")
m.optimize()

for var in m.getVars(): # descision variable
    print(var.varName, '=', var.x, var.obj)

#for con in m.getConstrs(): # constraints
#    print(con.ConstrName, ': slack =', con.slack,', shadow price=',
#          con.pi,',', (con.RHS, con.SARHSLow, con.SARHSUp))
    
print("objective value =", m.objVal)

(9,) (6, 9) (6,)
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x9cd6d993
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 3e+02]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)
Found heuristic solution: objective 5725.0000000
Found heuristic solution: objective 4600.0000000
Found heuristic solution: objective 4525.0000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      4525.00000 4525.00000  0.00%     - 

## Problem Example: C6Q4
Solve the following LP problem:

\begin{align}
&\text{min}\\
&\qquad z=14x_{A1} + 9x_{A2} + 16x_{A3} + 18x_{A4}\\
&\qquad ~~+ 11x_{B1} + 8x_{B2} + 100x_{B3} + 16x_{B4}\\
&\qquad ~~+ 16x_{C1} + 12x_{C2} + 10x_{C3} + 22x_{C4}\\
&\text{subject to}\\
&\qquad x_{A1} +x_{A2} +x_{A3} +x_{A4} \le 150 \\
&\qquad x_{B1} +x_{B2} +x_{B3} +x_{B4} \le 210\\ 
&\qquad x_{C1} +x_{C2} +x_{C3} +x_{C4} \le 320\\
&\qquad x_{A1} +x_{B1} +x_{C1} =130 \\
&\qquad x_{A2} +x_{B2} +x_{C2} =70 \\
&\qquad x_{A3} +x_{B3} +x_{C3} =180\\
&\qquad x_{A4} +x_{B4} +x_{C4} =240\\
&\qquad x_{ij} \ge 0~ \text{and integer}\\
\end{align}  


In [8]:
c = [14,9,16,18,11,8,1000,16,16,12,10,22]    
A = [[1,1,1,1,0,0,0,0,0,0,0,0],
     [0,0,0,0,1,1,1,1,0,0,0,0],
     [0,0,0,0,0,0,0,0,1,1,1,1],
     [1,0,0,0,1,0,0,0,1,0,0,0],
     [0,1,0,0,0,1,0,0,0,1,0,0],
     [0,0,1,0,0,0,1,0,0,0,1,0],
     [0,0,0,1,0,0,0,1,0,0,0,1]]
b =  [150,210,320,130,70,180,240]
print(np.array(c).shape,np.array(A).shape,np.array(b).shape)
decision_variables = range(len(c))     
constraints = range(np.array(A).shape[0])

m = Model("example")

x = []
for i in decision_variables:
    x.append(m.addVar(lb = 0, vtype = GRB.INTEGER, name = 'x' + str(i)))

m.setObjective(quicksum(c[i] * x[i] for i in decision_variables) , GRB.MINIMIZE) 

m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           <= b[j] for j in constraints[:3]), "constraints")

m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           == b[j] for j in constraints[3:]), "constraints")
              
m.optimize()

for var in m.getVars(): # descision variable
    print(var.varName, '=', var.x, var.obj)

#for con in m.getConstrs(): # constraints
#    print(con.ConstrName, ': slack =', con.slack,', shadow price=',
#          con.pi,',', (con.RHS, con.SARHSLow, con.SARHSUp))
    
print("objective value =", m.objVal)

(12,) (7, 12) (7,)
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xe2efa14f
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 3e+02]
Found heuristic solution: objective 8450.0000000
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (0 binary)
Found heuristic solution: objective 8444.0000000

Root relaxation: objective 8.260000e+03, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    8260.0000000 8260.00000  0.00%     -    0s

Explored 1 nodes (6 

In [9]:
c = [14,9,16,18,11,8,1000,16,16,12,10,22]    
A = [[1,1,1,1,0,0,0,0,0,0,0,0],
     [0,0,0,0,1,1,1,1,0,0,0,0],
     [0,0,0,0,0,0,0,0,1,1,1,1],
     [1,0,0,0,1,0,0,0,1,0,0,0],
     [0,1,0,0,0,1,0,0,0,1,0,0],
     [0,0,1,0,0,0,1,0,0,0,1,0],
     [0,0,0,1,0,0,0,1,0,0,0,1]]
b =  [150,210,290,130,70,180,240]
print(np.array(c).shape,np.array(A).shape,np.array(b).shape)
decision_variables = range(len(c))     
constraints = range(np.array(A).shape[0])

m = Model("example")

x = []
for i in decision_variables:
    x.append(m.addVar(lb = 0, vtype = GRB.INTEGER, name = 'x' + str(i)))

m.setObjective(quicksum(c[i] * x[i] for i in decision_variables) , GRB.MINIMIZE) 

m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           <= b[j] for j in constraints[:3]), "constraints")

m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           == b[j] for j in constraints[3:]), "constraints")
              
m.optimize()

for var in m.getVars(): # descision variable
    print(var.varName, '=', var.x, var.obj)

#for con in m.getConstrs(): # constraints
#    print(con.ConstrName, ': slack =', con.slack,', shadow price=',
#          con.pi,',', (con.RHS, con.SARHSLow, con.SARHSUp))
    
print("objective value =", m.objVal)

(12,) (7, 12) (7,)
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x65203382
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 3e+02]
Found heuristic solution: objective 28310.000000
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (0 binary)
Found heuristic solution: objective 27321.000000

Root relaxation: objective 8.260000e+03, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    8260.0000000 8260.00000  0.00%     -    0s

Explored 1 nodes (6 

## Problem Example: C6Q30
Transshipment probelm is an extension of the transportation model in which intermediate trans-shipment points are added between sources and destinations.

In [10]:
c = [420,390,610,510,590,470,450,360,380,75,63,81,125,110,95,68,82,95]    
A = [[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0],
     [0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0],
     [0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1],
     [1,0,0,1,0,0,1,0,0,-1,-1,-1,0,0,0,0,0,0],
     [0,1,0,0,1,0,0,1,0,0,0,0,-1,-1,-1,0,0,0],
     [0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,-1,-1,-1]]
b =  [55,78,37,60,45,50,0,0,0]
print(np.array(c).shape,np.array(A).shape,np.array(b).shape)
decision_variables = range(len(c))     
constraints = range(np.array(A).shape[0])

m = Model("example")

x = []
for i in decision_variables:
    x.append(m.addVar(lb = 0, vtype = GRB.INTEGER, name = 'x' + str(i)))

m.setObjective(quicksum(c[i] * x[i] for i in decision_variables) , GRB.MINIMIZE) 

m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           <= b[j] for j in constraints[:3]), "constraints")

m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           == b[j] for j in constraints[3:]), "constraints")
              
m.optimize()

for var in m.getVars(): # descision variable
    print(var.varName, '=', var.x, var.obj)

#for con in m.getConstrs(): # constraints
#    print(con.ConstrName, ': slack =', con.slack,', shadow price=',
#          con.pi,',', (con.RHS, con.SARHSLow, con.SARHSUp))
    
print("objective value =", m.objVal)

(18,) (9, 18) (9,)
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 9 rows, 18 columns and 36 nonzeros
Model fingerprint: 0x6661d933
Variable types: 0 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 8e+01]
Found heuristic solution: objective 89740.000000
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 36 nonzeros
Variable types: 0 continuous, 18 integer (0 binary)
Found heuristic solution: objective 89600.000000

Root relaxation: objective 7.736200e+04, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    77362.000000 77362.0000  0.00%     -    0s

Explored 1 nodes (1

##Problem Example: C6Q43
Solve the following LP problem:

\begin{align}
&\text{min}\\
&\qquad z=12x_{1A} + 11x_{1B} + 8x_{1C} + 14x_{1D}\\
&\qquad~~+ 10x_{2A} + 9x_{2B} + 10x_{2C} + 8x_{2D}\\
&\qquad~~+ 14x_{3A} + 100x_{3B} + 7x_{3C} + 11x_{3D}\\ 
&\qquad~~+ 6x_{4A} + 8x_{4B} + 10x_{4C} + 9x_{4D}\\
&\text{subject to}\\
&\qquad x_{1A} +x_{1B} +x_{1C} +x_{1D} =1 \\
&\qquad x_{2A} +x_{2B} +x_{2C} +x_{2D} =1 \\
&\qquad x_{3A} +x_{3B} +x_{3C} +x_{3D} =1 \\
&\qquad x_{4A} +x_{4B} +x_{4C} +x_{4D} =1 \\
&\qquad x_{1A} +x_{2A} +x_{3A} +x_{4A} =1 \\
&\qquad x_{1B} +x_{2B} +x_{3B} +x_{4B} =1 \\
&\qquad x_{1C} +x_{2C} +x_{3C} +x_{4C} =1 \\
&\qquad x_{1D} +x_{2D} +x_{3D} +x_{4D} =1 \\
&\qquad x_{ij} \ge 0~ \text{and integer}\\
\end{align} 

In [11]:
c = [12,11,8,14,10,9,10,8,14,1000,7,11,6,8,10,9]    
A = [[1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
     [0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0],
     [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1],
     [1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0],
     [0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0],
     [0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0],
     [0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1]]
b =  [1,1,1,1,1,1,1,1]
print(np.array(c).shape,np.array(A).shape,np.array(b).shape)
decision_variables = range(len(c))     
constraints = range(np.array(A).shape[0])

m = Model("example")

x = []
for i in decision_variables:
    x.append(m.addVar(lb = 0, vtype = GRB.INTEGER, name = 'x' + str(i)))

m.setObjective(quicksum(c[i] * x[i] for i in decision_variables) , GRB.MINIMIZE) 

m.addConstrs((quicksum(A[j][i] * x[i] for i in decision_variables) 
                           == b[j] for j in constraints), "constraints")
           
m.optimize()

for var in m.getVars(): # descision variable
    print(var.varName, '=', var.x, var.obj)

#for con in m.getConstrs(): # constraints
#    print(con.ConstrName, ': slack =', con.slack,', shadow price=',
#          con.pi,',', (con.RHS, con.SARHSLow, con.SARHSUp))
    
print("objective value =", m.objVal)

(16,) (8, 16) (8,)
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0xe843bdde
Variable types: 0 continuous, 16 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1027.0000000
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 3.200000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      32.0000000   32.00000  0.00%     -    0s

Explored 1 nodes (6 simplex iterations) in 0.04 seconds (0.00 work u